## **Problem Statement**

### **Business Context**

An automobile dealership in Los Vegas specializes in selling luxury and non-luxury vehicles. They cater to diverse customer preferences with varying vehicle specifications, such as mileage, engine capacity, and seating capacity. However, the dealership faces significant challenges in maintaining consistency and efficiency across its pricing strategy due to reliance on manual processes and disconnected systems. Pricing evaluations are prone to errors, updates are delayed, and scaling operations are difficult as demand grows. These inefficiencies impact revenue and customer trust. Recognizing the need for a reliable and scalable solution, the dealership is seeking to implement a unified system that ensures seamless integration of data-driven pricing decisions, adaptability to changing market conditions, and operational efficiency.

### **Objective**

The dealership has hired you as an MLOps Engineer to design and implement an MLOps pipeline that automates the pricing workflow. This pipeline will encompass data cleaning, preprocessing, transformation, model building, training, evaluation, and registration with CI/CD capabilities to ensure continuous integration and delivery. Your role is to overcome challenges such as integrating disparate data sources, maintaining consistent model performance, and enabling scalable, automated updates to meet evolving business needs. The expected outcomes are a robust, automated system that improves pricing accuracy, operational efficiency, and scalability, driving increased profitability and customer satisfaction.

### **Data Description**

The dataset contains attributes of used cars sold in various locations. These attributes serve as key data points for CarOnSell's pricing model. The detailed attributes are:

- **Segment:** Describes the category of the vehicle, indicating whether it is a luxury or non-luxury segment.

- **Kilometers_Driven:** The total number of kilometers the vehicle has been driven.

- **Mileage:** The fuel efficiency of the vehicle, measured in kilometers per liter (km/l).

- **Engine:** The engine capacity of the vehicle, measured in cubic centimeters (cc). 

- **Power:** The power of the vehicle's engine, measured in brake horsepower (BHP). 

- **Seats:** The number of seats in the vehicle, can influence the vehicle's classification, usage, and pricing based on customer needs.

- **Price:** The price of the vehicle, listed in lakhs (units of 100,000), represents the cost to the consumer for purchasing the vehicle.

# **GItHub Repo Link**

In [ ]:
https://github.com/rakshit2711/cars

## **1. AzureML Environment Setup and Data Preparation**

### **1.1 Connect to Azure Machine Learning Workspace**

Installing dependencies

In [1]:
!pip install azure-ai-ml azure-identity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 60.1 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.1/264.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.1/102.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.3/183.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.2/291.2 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 5.7 MB/s eta 0:00:00
  Attempting uninstal

In [3]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()

In [4]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="f2f9c878-1ca2-40dc-a868-b68e6b45074d",  # Replace with your Azure subscription ID
    resource_group_name="default_resource_group",  # Replace with your resource group name
    workspace_name="cars",  # Replace with your ML workspace name
)

### **1.2 Set Up Compute Cluster**

In [5]:
from azure.ai.ml.entities import AmlCompute

# Name assigned to the compute cluster
cpu_compute_target = "cars"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")

    # Let's create the Azure ML compute object with the intended parameters
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        # Azure ML Compute is the on-demand VM service
        type="amlcompute",
        # VM Family
        size="Standard_DS11_v2",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=1,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=180,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )

    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster).result()

print(
    f"AMLCompute with name {cpu_cluster.name} is created, the compute size is {cpu_cluster.size}"
)

You already have a cluster named cars, we'll reuse it as is.
AMLCompute with name cars is created, the compute size is Standard_DS12_v2


### **1.3 Register Dataset as Data Asset**

In [6]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# Path to the local dataset
local_data_path = 'used_cars.csv'  # Updated filename to match workspace

# Create and register the dataset as an AzureML data asset
data_asset = Data(
    path=local_data_path,
    type=AssetTypes.URI_FILE, 
    description="A dataset of used cars for price prediction",
    name="used-cars-data"
)

In [7]:
ml_client.data.create_or_update(data_asset)

Data({'path': 'azureml://subscriptions/f2f9c878-1ca2-40dc-a868-b68e6b45074d/resourcegroups/default_resource_group/workspaces/cars/datastores/workspaceblobstore/paths/LocalUpload/0b8e06a9f14bf45a52b1c21394f1cdf03017517cd48663b3e20a05882ff35cdd/used_cars.csv', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'used-cars-data', 'description': 'A dataset of used cars for price prediction', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/f2f9c878-1ca2-40dc-a868-b68e6b45074d/resourceGroups/default_resource_group/providers/Microsoft.MachineLearningServices/workspaces/cars/data/used-cars-data/versions/2', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cars/code/Users/Rakshit_1746373710099', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7c0e1443dbd0>, '

### **1.4 Create and Configure Job Environment**

In [26]:
# Create a directory for the preprocessing script
import os

src_dir_env = "./env"
os.makedirs(src_dir_env, exist_ok=True)

In [27]:
%%writefile {src_dir_env}/conda.yml
name: sklearn-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - pip=21.2.4
  - scikit-learn=0.23.2
  - scipy=1.7.1
  - pip:  
    - mlflow==2.8.1
    - azureml-mlflow==1.51.0
    - azureml-inference-server-http
    - azureml-core==1.49.0
    - cloudpickle==1.6.0

Writing ./env/conda.yml


In [10]:
from azure.ai.ml.entities import Environment, BuildContext

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="env/conda.yml",
    name="machine_learning_E2E",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'machine_learning_E2E', 'description': 'Environment created from a Docker image plus Conda environment.', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/f2f9c878-1ca2-40dc-a868-b68e6b45074d/resourceGroups/default_resource_group/providers/Microsoft.MachineLearningServices/workspaces/cars/environments/machine_learning_E2E/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cars/code/Users/Rakshit_1746373710099', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7c0df28f3550>, 'serialize': <msrest.serialization.Serializer object at 0x7c0e2c10b970>, 'version': '1', 'conda_file': {'channels': ['conda-for

## **2. Model Development Workflow**

### **2.1 Data Preparation**

This **Data Preparation job** is designed to process an input dataset by splitting it into two parts: one for training the model and the other for testing it. The script accepts three inputs: the location of the input data (`used_cars.csv`), the ratio for splitting the data into training and testing sets (`test_train_ratio`), and the paths to save the resulting training (`train_data`) and testing (`test_data`) data. The script first reads the input CSV data from a data asset URI, then splits it using Scikit-learn's train_test_split function, and saves the two parts to the specified directories. It also logs the number of records in both the training and testing datasets using MLflow.

In [28]:
# Create a directory for the data preparation script
import os

src_dir = "./src"
os.makedirs(src_dir, exist_ok=True)

In [30]:
%%writefile {src_dir}/data_prep.py
"""
Data Preparation Script for Used Cars Price Prediction
This script handles data loading, cleaning, and splitting for the MLOps pipeline.
"""

import argparse
import logging
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import mlflow

def main():
    """Main function to execute data preparation"""
    
    # Input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="path to input data")
    parser.add_argument("--test_train_ratio", type=float, required=False, default=0.25)
    parser.add_argument("--train_data", type=str, help="path to train data")
    parser.add_argument("--test_data", type=str, help="path to test data")
    args = parser.parse_args()
    
    # Start Logging
    mlflow.start_run()
    
    print("Input Data:", args.data)
    print("Test/Train Ratio:", args.test_train_ratio)
    
    # Read the data
    print("Reading data...")
    all_data = pd.read_csv(args.data)
    
    print(f"Dataset shape: {all_data.shape}")
    print("Dataset info:")
    print(all_data.info())
    print("\nDataset description:")
    print(all_data.describe())
    
    # Check for missing values
    print("\nMissing values:")
    print(all_data.isnull().sum())
    
    # Data preprocessing
    print("\nStarting data preprocessing...")
    
    # Handle missing values if any
    if all_data.isnull().sum().sum() > 0:
        print("Handling missing values...")
        # For numeric columns, fill with median
        numeric_cols = all_data.select_dtypes(include=['float64', 'int64']).columns
        for col in numeric_cols:
            if all_data[col].isnull().sum() > 0:
                all_data[col].fillna(all_data[col].median(), inplace=True)
        
        # For categorical columns, fill with mode
        categorical_cols = all_data.select_dtypes(include=['object']).columns
        for col in categorical_cols:
            if all_data[col].isnull().sum() > 0:
                all_data[col].fillna(all_data[col].mode()[0], inplace=True)
    
    # Remove any duplicates
    initial_rows = len(all_data)
    all_data = all_data.drop_duplicates()
    print(f"Removed {initial_rows - len(all_data)} duplicate rows")
    
    # Convert column names to lowercase for consistency
    all_data.columns = all_data.columns.str.lower()
    
    # Split the data into train and test sets
    print("Splitting data into train and test sets...")
    train_df, test_df = train_test_split(
        all_data,
        test_size=args.test_train_ratio,
        random_state=42,
        stratify=all_data['segment']  # Stratify by segment to maintain distribution
    )
    
    # Create the output directories
    os.makedirs(args.train_data, exist_ok=True)
    os.makedirs(args.test_data, exist_ok=True)
    
    # Save the train and test sets
    train_df.to_csv(os.path.join(args.train_data, "train.csv"), index=False)
    test_df.to_csv(os.path.join(args.test_data, "test.csv"), index=False)
    
    # Log key metrics
    mlflow.log_metric("total_samples", len(all_data))
    mlflow.log_metric("train_samples", len(train_df))
    mlflow.log_metric("test_samples", len(test_df))
    mlflow.log_metric("train_test_ratio", args.test_train_ratio)
    
    print(f"Total samples: {len(all_data)}")
    print(f"Training samples: {len(train_df)}")
    print(f"Testing samples: {len(test_df)}")
    print(f"Test ratio: {args.test_train_ratio}")
    
    # Log data distribution insights
    print("\nSegment distribution in training data:")
    segment_dist = train_df['segment'].value_counts()
    print(segment_dist)
    
    for segment, count in segment_dist.items():
        mlflow.log_metric(f"train_{segment.replace(' ', '_')}_count", count)
    
    print("\nData preparation completed successfully!")
    
    # Stop Logging
    mlflow.end_run()

if __name__ == "__main__":
    main()

Overwriting ./src/data_prep.py


#### **Define Data Preparation job**

For this AzureML job, we define the `command` object that takes input files and output directories, then executes the script with the provided inputs and outputs. The job runs in a pre-configured AzureML environment with the necessary libraries. The result will be two separate datasets for training and testing, ready for use in subsequent steps of the machine learning pipeline.

In [13]:
from azure.ai.ml import command, Input, Output

# Get the data asset
data_asset = ml_client.data.get(name="used-cars-data", version="1")

# Define the data preparation job
data_prep_job = command(
    inputs=dict(
        data=Input(type="uri_file", path=data_asset.path),
        test_train_ratio=0.2,
    ),
    outputs=dict(
        train_data=Output(type="uri_folder", mode="rw_mount"),
        test_data=Output(type="uri_folder", mode="rw_mount"),
    ),
    code="./src",  # location of source code
    command="python data_prep.py --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} --train_data ${{outputs.train_data}} --test_data ${{outputs.test_data}}",
    environment="machine_learning_E2E@latest",
    compute="cars",
    display_name="data_preparation",
    description="Data preparation for used cars price prediction",
)

print("Data preparation job defined successfully!")

Data preparation job defined successfully!


### **2.2 Training the Model**

This Model Training job is designed to train a **Random Forest Regressor** on the dataset that was split into training and testing sets in the previous data preparation job. This job script accepts five inputs: the path to the training data (`train_data`), the path to the testing data (`test_data`), the number of trees in the forest (`n_estimators`, with a default value of 100), the maximum depth of the trees (`max_depth`, which is set to None by default), and the path to save the trained model (`model_output`).

The script begins by reading the training and testing data files, then processes the data to separate features (X) and target labels (y). A Random Forest Regressor model is initialized using the given n_estimators and max_depth, and it is trained using the training data. The model's performance is evaluated using the `Mean Squared Error (MSE)`. The MSE score is logged in MLflow. Finally, the trained model is saved and stored in the specified output location as an MLflow model. The job completes by logging the final MSE score and ending the MLflow run.


In [31]:
%%writefile {src_dir}/train.py
"""
Model Training Script for Used Cars Price Prediction
This script handles model training with hyperparameter tuning and evaluation.
"""

import argparse
import logging
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import mlflow
import mlflow.sklearn
import joblib

def preprocess_features(df, label_encoders=None, scaler=None, is_training=True):
    """Preprocess features for training or prediction"""
    
    # Separate features and target
    if 'price' in df.columns:
        X = df.drop(['price'], axis=1)
        y = df['price']
    else:
        X = df.copy()
        y = None
    
    # Handle categorical variables
    categorical_cols = ['segment']
    
    if is_training:
        label_encoders = {}
        for col in categorical_cols:
            if col in X.columns:
                le = LabelEncoder()
                X[col] = le.fit_transform(X[col].astype(str))
                label_encoders[col] = le
        
        # Scale numerical features
        numerical_cols = ['kilometers_driven', 'mileage', 'engine', 'power', 'seats']
        scaler = StandardScaler()
        X[numerical_cols] = scaler.fit_transform(X[numerical_cols])
        
    else:
        # Apply existing encoders and scaler
        for col in categorical_cols:
            if col in X.columns and col in label_encoders:
                X[col] = label_encoders[col].transform(X[col].astype(str))
        
        numerical_cols = ['kilometers_driven', 'mileage', 'engine', 'power', 'seats']
        X[numerical_cols] = scaler.transform(X[numerical_cols])
    
    return X, y, label_encoders, scaler

def main():
    """Main function to execute model training"""
    
    # Input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--train_data", type=str, help="path to train data")
    parser.add_argument("--test_data", type=str, help="path to test data")
    parser.add_argument("--n_estimators", type=int, required=False, default=100)
    parser.add_argument("--max_depth", type=int, required=False, default=None)
    parser.add_argument("--model_output", type=str, help="path to model file")
    args = parser.parse_args()
    
    # Start Logging
    mlflow.start_run()
    
    # Log parameters
    mlflow.log_param("n_estimators", args.n_estimators)
    mlflow.log_param("max_depth", args.max_depth)
    
    print("Loading training data...")
    train_df = pd.read_csv(os.path.join(args.train_data, "train.csv"))
    
    print("Loading testing data...")
    test_df = pd.read_csv(os.path.join(args.test_data, "test.csv"))
    
    print(f"Training data shape: {train_df.shape}")
    print(f"Testing data shape: {test_df.shape}")
    
    # Preprocess the data
    print("Preprocessing training data...")
    X_train, y_train, label_encoders, scaler = preprocess_features(train_df, is_training=True)
    
    print("Preprocessing testing data...")
    X_test, y_test, _, _ = preprocess_features(test_df, label_encoders, scaler, is_training=False)
    
    print(f"Training features shape: {X_train.shape}")
    print(f"Testing features shape: {X_test.shape}")
    
    # Initialize and train the model
    print("Training Random Forest model...")
    max_depth = args.max_depth if args.max_depth != -1 else None
    
    model = RandomForestRegressor(
        n_estimators=args.n_estimators,
        max_depth=max_depth,
        random_state=42,
        n_jobs=-1
    )
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    print("Making predictions...")
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)
    
    # Calculate metrics
    train_mse = mean_squared_error(y_train, train_predictions)
    test_mse = mean_squared_error(y_test, test_predictions)
    train_r2 = r2_score(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    train_mae = mean_absolute_error(y_train, train_predictions)
    test_mae = mean_absolute_error(y_test, test_predictions)
    
    # Log metrics
    mlflow.log_metric("train_mse", train_mse)
    mlflow.log_metric("test_mse", test_mse)
    mlflow.log_metric("train_r2", train_r2)
    mlflow.log_metric("test_r2", test_r2)
    mlflow.log_metric("train_mae", train_mae)
    mlflow.log_metric("test_mae", test_mae)
    
    print(f"Training MSE: {train_mse:.4f}")
    print(f"Testing MSE: {test_mse:.4f}")
    print(f"Training R²: {train_r2:.4f}")
    print(f"Testing R²: {test_r2:.4f}")
    print(f"Training MAE: {train_mae:.4f}")
    print(f"Testing MAE: {test_mae:.4f}")
    
    # Create the output directory
    os.makedirs(args.model_output, exist_ok=True)
    
    # Save the model and preprocessors
    model_path = os.path.join(args.model_output, "model.pkl")
    joblib.dump(model, model_path)
    
    preprocessors_path = os.path.join(args.model_output, "preprocessors.pkl")
    preprocessors = {
        'label_encoders': label_encoders,
        'scaler': scaler
    }
    joblib.dump(preprocessors, preprocessors_path)
    
    # Log the model
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        signature=None
    )
    
    print("Model training completed successfully!")
    
    # Stop Logging
    mlflow.end_run()

if __name__ == "__main__":
    main()

Writing ./src/train.py


#### **Define Model Training Job**

For this AzureML job, we define the `command` object that takes the paths to the training and testing data, the number of trees in the forest (`n_estimators`), and the maximum depth of the trees (`max_depth`) as inputs, and outputs the trained model. The command runs in a pre-configured AzureML environment with all the necessary libraries. The job produces a trained **Random Forest Regressor model**, which can be used for predicting the price of used cars based on the given attributes.

In [15]:
# Define the training job
train_job = command(
    inputs=dict(
        train_data=Input(type="uri_folder"),
        test_data=Input(type="uri_folder"),
        n_estimators=100,
        max_depth=-1,  # -1 represents None
    ),
    outputs=dict(
        model_output=Output(type="uri_folder", mode="rw_mount"),
    ),
    code="./src",  # location of source code
    command="python train.py --train_data ${{inputs.train_data}} --test_data ${{inputs.test_data}} --n_estimators ${{inputs.n_estimators}} --max_depth ${{inputs.max_depth}} --model_output ${{outputs.model_output}}",
    environment="machine_learning_E2E@latest",
    compute="cars",
    display_name="train_model",
    description="Train Random Forest model for used cars price prediction",
)

print("Training job defined successfully!")

Training job defined successfully!


### **2.3 Registering the Best Trained Model**

The **Model Registration job** is designed to take the best-trained model from the hyperparameter tuning sweep job and register it in MLflow as a versioned artifact for future use in the used car price prediction pipeline. This job script accepts one input: the path to the trained model (model). The script begins by loading the model using the `mlflow.sklearn.load_model()` function. Afterward, it registers the model in the MLflow model registry, assigning it a descriptive name (`used_cars_price_prediction_model`) and specifying an artifact path (`random_forest_price_regressor`) where the model artifacts will be stored. Using MLflow's `log_model()` function, the model is logged along with its metadata, ensuring that the model is easily trackable and retrievable for future evaluation, deployment, or retraining.

In [32]:
%%writefile {src_dir}/model_register.py
"""
Model Registration Script for Used Cars Price Prediction
This script registers the best trained model in MLflow registry.
"""

import argparse
import os
import mlflow
import mlflow.sklearn
import joblib

def main():
    """Main function to execute model registration"""
    
    # Input arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", type=str, help="path to model file")
    args = parser.parse_args()
    
    # Start Logging
    mlflow.start_run()
    
    print(f"Loading model from: {args.model}")
    
    # Load the model
    model_path = os.path.join(args.model, "model.pkl")
    model = joblib.load(model_path)
    
    # Load preprocessors
    preprocessors_path = os.path.join(args.model, "preprocessors.pkl")
    preprocessors = joblib.load(preprocessors_path)
    
    print("Model loaded successfully!")
    print(f"Model type: {type(model)}")
    
    # Register the model in MLflow
    print("Registering model in MLflow...")
    
    # Log the model with all artifacts
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="random_forest_price_regressor",
        registered_model_name="used_cars_price_prediction_model",
        signature=None,  # You can add model signature here for better tracking
        input_example=None  # You can add input example here
    )
    
    # Log preprocessors as artifacts
    mlflow.log_artifact(preprocessors_path, "preprocessors")
    
    print("Model registered successfully in MLflow!")
    print("Model name: used_cars_price_prediction_model")
    print("Artifact path: random_forest_price_regressor")
    
    # Stop Logging
    mlflow.end_run()

if __name__ == "__main__":
    main()

Writing ./src/model_register.py


#### **Define Model Register Job**

For this AzureML job, a `command` object is defined to execute the `model_register.py` script. It accepts the best-trained model as input, runs the script in the `AzureML-sklearn-1.0-ubuntu20.04-py38-cpu` environment, and uses the same compute cluster as the previous jobs (`cars`). This job plays a crucial role in the pipeline by ensuring that the best-performing model identified during hyperparameter tuning is systematically stored and made available in the MLflow registry for further evaluation, deployment, or retraining. Integrating this job into the end-to-end pipeline automates the process of registering high-quality models, completing the model development lifecycle and enabling the prediction of used car prices.

In [17]:
# Define the model registration job
register_job = command(
    inputs=dict(
        model=Input(type="uri_folder"),
    ),
    code="./src",  # location of source code
    command="python model_register.py --model ${{inputs.model}}",
    environment="machine_learning_E2E@latest",
    compute="cars",
    display_name="register_model",
    description="Register the trained model in MLflow registry",
)

print("Model registration job defined successfully!")

Model registration job defined successfully!


### **2.4. Assembling the End-to-End Workflow**

The end-to-end pipeline integrates all the previously defined jobs into a seamless workflow, automating the process of data preparation, model training, hyperparameter tuning, and model registration. The pipeline is designed using Azure Machine Learning's `@pipeline` decorator, specifying the compute target and providing a detailed description of the workflow.

In [18]:
from azure.ai.ml import dsl

@dsl.pipeline(
    compute="cars",
    description="End-to-End MLOps Pipeline for Used Cars Price Prediction",
)
def used_cars_pipeline(
    pipeline_data,
    test_train_ratio=0.2,
    n_estimators=100,
    max_depth=-1,
):
    """
    End-to-end pipeline for used cars price prediction
    
    Args:
        pipeline_data: Input dataset
        test_train_ratio: Ratio for train-test split
        n_estimators: Number of trees in Random Forest
        max_depth: Maximum depth of trees
    """
    
    # Step 1: Data Preparation
    data_prep_step = data_prep_job(
        data=pipeline_data,
        test_train_ratio=test_train_ratio,
    )
    
    # Step 2: Model Training
    train_step = train_job(
        train_data=data_prep_step.outputs.train_data,
        test_data=data_prep_step.outputs.test_data,
        n_estimators=n_estimators,
        max_depth=max_depth,
    )
    
    # Step 3: Model Registration
    register_step = register_job(
        model=train_step.outputs.model_output,
    )
    
    # Return outputs
    return {
        "train_data": data_prep_step.outputs.train_data,
        "test_data": data_prep_step.outputs.test_data,
        "model": train_step.outputs.model_output,
    }

# Create pipeline instance
pipeline = used_cars_pipeline(
    pipeline_data=Input(type="uri_file", path=data_asset.path),
    test_train_ratio=0.2,
    n_estimators=100,
    max_depth=-1,
)

print("Pipeline defined successfully!")
print("Pipeline components:")
print("1. Data Preparation - Splits data into train/test sets")
print("2. Model Training - Trains Random Forest model")
print("3. Model Registration - Registers model in MLflow")

Pipeline defined successfully!
Pipeline components:
1. Data Preparation - Splits data into train/test sets
2. Model Training - Trains Random Forest model
3. Model Registration - Registers model in MLflow


## **3. Pipeline Execution and Monitoring**

### **3.1 Submit and Run the Pipeline**

In [19]:
# Submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    experiment_name="used-cars-price-prediction"
)

print(f"Pipeline job submitted with ID: {pipeline_job.name}")
print(f"Pipeline status: {pipeline_job.status}")
print(f"Pipeline URL: {pipeline_job.services['Studio'].endpoint}")

# You can uncomment the following line to wait for the pipeline to complete
# ml_client.jobs.stream(pipeline_job.name)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
pathOnCompute is not a known attribute

Pipeline job submitted with ID: calm_bean_8qrbm1d85q
Pipeline status: NotStarted
Pipeline URL: https://ml.azure.com/runs/calm_bean_8qrbm1d85q?wsid=/subscriptions/f2f9c878-1ca2-40dc-a868-b68e6b45074d/resourcegroups/default_resource_group/workspaces/cars&tid=a2799098-ec71-4199-a883-6274017f5282


## **4. CI/CD with GitHub Actions**

### **4.1 Setting up GitHub Repository**

This section demonstrates how to set up a GitHub repository with CI/CD capabilities using GitHub Actions for our MLOps pipeline. The workflow will automatically trigger the Azure ML pipeline when code changes are pushed to the repository.

In [20]:
# Create GitHub Actions workflow
import os

# Create .github/workflows directory
workflows_dir = "./.github/workflows"
os.makedirs(workflows_dir, exist_ok=True)

In [34]:
%%writefile {workflows_dir}/mlops-pipeline.yml
name: MLOps Pipeline - Used Cars Price Prediction

on:
  push:
    branches: [ main, develop ]
  pull_request:
    branches: [ main ]
  workflow_dispatch:

env:
  AZURE_ML_WORKSPACE: your-ml-workspace
  AZURE_RESOURCE_GROUP: your-resource-group
  AZURE_SUBSCRIPTION_ID: your-subscription-id

jobs:
  data-validation:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v3
    
    - name: Set up Python
      uses: actions/setup-python@v4
      with:
        python-version: '3.8'
    
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install pandas scikit-learn pytest
    
    - name: Run data validation tests
      run: |
        python -m pytest tests/ -v

  model-training:
    needs: data-validation
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v3
    
    - name: Set up Python
      uses: actions/setup-python@v4
      with:
        python-version: '3.8'
    
    - name: Install Azure CLI and ML extension
      run: |
        curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
        az extension add -n ml
    
    - name: Azure Login
      uses: azure/login@v1
      with:
        creds: ${{ secrets.AZURE_CREDENTIALS }}
    
    - name: Install Python dependencies
      run: |
        python -m pip install --upgrade pip
        pip install azure-ai-ml azure-identity

    - name: Run MLOps Pipeline
      run: |
        python .github/workflows/run_pipeline.py
      env:
        AZURE_SUBSCRIPTION_ID: ${{ env.AZURE_SUBSCRIPTION_ID }}
        AZURE_RESOURCE_GROUP: ${{ env.AZURE_RESOURCE_GROUP }}
        AZURE_ML_WORKSPACE: ${{ env.AZURE_ML_WORKSPACE }}

  model-deployment:
    needs: model-training
    runs-on: ubuntu-latest
    if: github.ref == 'refs/heads/main'
    steps:
    - uses: actions/checkout@v3
    
    - name: Set up Python
      uses: actions/setup-python@v4
      with:
        python-version: '3.8'
    
    - name: Azure Login
      uses: azure/login@v1
      with:
        creds: ${{ secrets.AZURE_CREDENTIALS }}
    
    - name: Deploy Model
      run: |
        echo "Model deployment step - implement based on your deployment strategy"
        # Add your model deployment logic here

Overwriting ./.github/workflows/mlops-pipeline.yml


In [33]:
%%writefile {workflows_dir}/run_pipeline.py
"""
Pipeline runner script for GitHub Actions
This script submits the MLOps pipeline to Azure ML
"""

import os
from azure.ai.ml import MLClient, Input, dsl, command
from azure.identity import DefaultAzureCredential

def main():
    # Get environment variables
    subscription_id = os.environ["AZURE_SUBSCRIPTION_ID"]
    resource_group = os.environ["AZURE_RESOURCE_GROUP"] 
    workspace_name = os.environ["AZURE_ML_WORKSPACE"]
    
    # Initialize ML Client
    credential = DefaultAzureCredential()
    ml_client = MLClient(
        credential=credential,
        subscription_id=subscription_id,
        resource_group_name=resource_group,
        workspace_name=workspace_name,
    )
    
    print(f"Connected to workspace: {workspace_name}")
    
    # Get the data asset
    try:
        data_asset = ml_client.data.get(name="used-cars-data", version="1")
        print("Data asset found successfully")
    except Exception as e:
        print(f"Error getting data asset: {e}")
        return
    
    # Submit the pipeline (you would need to recreate the pipeline definition here
    # or import it from a separate module)
    print("Pipeline would be submitted here...")
    print("This is a placeholder for the actual pipeline submission logic")

if __name__ == "__main__":
    main()

Overwriting ./.github/workflows/run_pipeline.py


## **5. Business Insights and Recommendations**

### **5.1 Key Findings from the MLOps Pipeline**

Based on the implementation of this end-to-end MLOps pipeline for used car price prediction, here are the key insights and recommendations for the Las Vegas automobile dealership:

#### **Technical Insights:**

1. **Automated Data Processing**: The pipeline automatically handles data cleaning, preprocessing, and feature engineering, ensuring consistent data quality across all pricing evaluations.

2. **Model Performance**: The Random Forest Regressor provides robust predictions by considering multiple features like segment, mileage, engine capacity, power, and seating capacity.

3. **Scalability**: The Azure ML infrastructure allows for easy scaling as the dealership grows and processes more data.

#### **Business Benefits:**

1. **Improved Pricing Accuracy**: 
   - Systematic data-driven approach reduces manual pricing errors
   - Consistent evaluation criteria across all vehicles
   - Real-time price updates based on market conditions

2. **Operational Efficiency**:
   - Automated pipeline reduces manual intervention
   - Faster processing of new inventory
   - Standardized workflows across the organization

3. **Enhanced Customer Trust**:
   - Transparent, data-driven pricing methodology
   - Consistent pricing standards
   - Reduced price discrepancies

#### **Recommendations for Implementation:**

1. **Data Quality Management**:
   - Implement regular data validation checks
   - Establish data governance policies
   - Monitor for data drift and model performance degradation

2. **Continuous Improvement**:
   - Regular model retraining with new data
   - A/B testing for different pricing strategies
   - Integration of external market data (fuel prices, economic indicators)

3. **Business Integration**:
   - Train staff on the new automated system
   - Establish feedback loops for model improvement
   - Create dashboard for business stakeholders

4. **Risk Management**:
   - Implement model monitoring and alerting
   - Establish fallback procedures for system failures
   - Regular audits of pricing decisions

### **5.2 Expected Business Impact**

1. **Revenue Optimization**: More accurate pricing leads to better profit margins and competitive positioning
2. **Customer Satisfaction**: Consistent and fair pricing improves customer trust and satisfaction
3. **Operational Costs**: Reduced manual effort in pricing evaluations
4. **Market Responsiveness**: Ability to quickly adapt to market changes
5. **Scalability**: Infrastructure supports business growth without proportional increase in operational complexity


## **6. GitHub Repository**

### **GitHub Repository Link**

🔗 **GitHub Repository**: [Add your public GitHub repository link here]

The repository contains:
- Complete source code for the MLOps pipeline
- Data preparation, training, and registration scripts
- GitHub Actions workflow for CI/CD
- Test files for data validation
- Documentation and README

Make sure the repository is public and contains all the necessary files for the MLOps pipeline implementation.